In [1]:
import os
os.environ['SPARK_VERSION'] = '3.1'

In [3]:
import pydeequ
from pydeequ.profiles import *
from pydeequ.checks import *
from pydeequ.verification import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession\
    .builder\
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

:: loading settings :: url = jar:file:/nfs/env/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/karpov/.ivy2/cache
The jars for the packages stored in: /home/karpov/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bf2b66c2-95e8-499e-8880-a8715388ff91;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found org.scala-lang#scala-reflect;2.12.1 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1.1 i

In [5]:
df = spark.read.parquet("shop-clients.parquet")

In [6]:
df.show()

+--------------------+------+----------+----------+-----------+------------+---+------------+---------------+
|                  id|   sex|      name|   surname|second_name|passport_num|age|total_orders|           city|
+--------------------+------+----------+----------+-----------+------------+---+------------+---------------+
|0002baa3306b48f99...|female|      Соня|  Ермолова|  Сергеевич|  7070177511| 21|          23|         Липецк|
|00070f3a44b24c5da...|  male|  Анатолий|    Марков|Анатольевич|  8914742901| 22|          34|Санкт-Петербург|
|000c3230d39040749...|  male|     Павел|    Марков| Кириллович|  3800229978| 69|          45|           Омск|
|00160cd13a3047aaa...|female|Александра|  Дроздова| Кириллович|  6699696036| 48|          75|           Омск|
|00264b9c03c24d82a...|female|       Яна|  Равдеева|   Павлович|  5443568581| 46|          92|         Москва|
|0026f92043174b488...|female| Екатерина|    Дубина|   Олегович|  7731771480| 61|          69|       Новгород|
|00293705f

In [7]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- second_name: string (nullable = true)
 |-- passport_num: long (nullable = true)
 |-- age: integer (nullable = true)
 |-- total_orders: integer (nullable = true)
 |-- city: string (nullable = true)



# Analyzer

In [8]:
from pydeequ.analyzers import *

analyzer = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("name")) \
                    .addAnalyzer(Completeness("surname")) \
                    .addAnalyzer(Completeness("second_name")) \
                    .addAnalyzer(Completeness("passport_num")) \
                    .addAnalyzer(Completeness("city")) \
                    .addAnalyzer(Compliance("age less than 0", 'age<0')) \
                    .addAnalyzer(Compliance("age great than 100", 'age>100')) \
                    .addAnalyzer(Compliance("orders less than 0", 'total_orders<0')) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analyzer)
analysisResult_df.show()

+-------+------------------+------------+-------+
| entity|          instance|        name|  value|
+-------+------------------+------------+-------+
| Column|              city|Completeness| 0.9985|
| Column|      passport_num|Completeness|  0.999|
| Column|orders less than 0|  Compliance| 7.5E-4|
| Column|   age less than 0|  Compliance|    0.0|
| Column|       second_name|Completeness| 0.9995|
|Dataset|                 *|        Size|20000.0|
| Column|              name|Completeness|    1.0|
| Column|           surname|Completeness|    1.0|
| Column|age great than 100|  Compliance|  0.001|
+-------+------------------+------------+-------+



# Constraint Verification

In [9]:
check_age = Check(spark, CheckLevel.Warning, "Users Age Check")\
        .hasMax("age", lambda x: x <= 100.0)  \
        .isNonNegative("age") \
        .isComplete("age")
check_fio = Check(spark, CheckLevel.Warning, "Users FIO Check")\
        .isComplete("name") \
        .isComplete("surname") \
        .isComplete("second_name")
check_sex = Check(spark, CheckLevel.Warning, "Users Sex Check")\
        .isContainedIn("sex", ["male", "female"])
check_dataset = Check(spark, CheckLevel.Error, "Users Dataset Check")\
        .hasSize(lambda x: x >= 20000)\
        .isUnique("id")

Python Callback server started!


In [10]:
checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check_age) \
    .addCheck(check_fio) \
    .addCheck(check_sex) \
    .addCheck(check_dataset) \
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.toPandas().head(20)

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Users Age Check,Warning,Warning,"MaximumConstraint(Maximum(age,None))",Failure,Value: 18798.0 does not meet the constraint re...
1,Users Age Check,Warning,Warning,ComplianceConstraint(Compliance(age is non-neg...,Success,
2,Users Age Check,Warning,Warning,"CompletenessConstraint(Completeness(age,None))",Success,
3,Users FIO Check,Warning,Warning,"CompletenessConstraint(Completeness(name,None))",Success,
4,Users FIO Check,Warning,Warning,"CompletenessConstraint(Completeness(surname,No...",Success,
5,Users FIO Check,Warning,Warning,CompletenessConstraint(Completeness(second_nam...,Failure,Value: 0.9995 does not meet the constraint req...
6,Users Sex Check,Warning,Success,ComplianceConstraint(Compliance(sex contained ...,Success,
7,Users Dataset Check,Error,Success,SizeConstraint(Size(None)),Success,
8,Users Dataset Check,Error,Success,"UniquenessConstraint(Uniqueness(List(id),None))",Success,


In [11]:
VerificationResult.successMetricsAsDataFrame(spark, checkResult).toPandas().head(20)

,entity,instance,name,value
0,Column,age is non-negative,Compliance,1.0000
1,Column,second_name,Completeness,0.9995
2,Column,"sex contained in male,female",Compliance,1.0000
3,Dataset,*,Size,20000.0000
4,Column,id,Uniqueness,1.0000
5,Column,name,Completeness,1.0000
6,Column,surname,Completeness,1.0000
7,Column,age,Maximum,18798.0000
8,Column,age,Completeness,1.0000
